# Using Translation API

In [ ]:
!pip install --quiet evaluate
!pip install --quiet googletrans==3.1.0a0

In [ ]:
from googletrans import Translator
import pandas as pd
import torch
from datasets import load_dataset 
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification , AutoTokenizer , AutoModel
import torch.nn as nn
from tqdm.auto import tqdm
from transformers import DataCollatorWithPadding , get_scheduler
from torch.utils.data import DataLoader
import evaluate
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit , train_test_split
import numpy as np
from accelerate import Accelerator

device = torch.device ("cuda") if torch.cuda.is_available() else torch.device ("cpu" )
# vietnamese_text = "Xin chào, bạn đang làm gì?"
# english_translation = translate_vietnamese_to_english(vietnamese_text)
# print(english_translation)


# Translate + Model ENG

In [ ]:
dataset_link = "uitnlp/vietnamese_students_feedback"
from datasets import load_dataset
dataset = load_dataset(dataset_link)
dataset

### TRANSLATE + DELETE NEUTRAL + CONVERT LABEL

In [68]:
def translate_vietnamese_to_english(text):
    translator = Translator()
    translation = translator.translate(text, src='vi', dest='en')
    return translation.text

In [71]:
def del_neutral (samples) :
    return samples['sentiment'] == 0 or samples['sentiment'] == 2
no_neutral_data = dataset.filter (del_neutral)
no_neutral_data

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 10968
    })
    validation: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 1510
    })
    test: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 2999
    })
})

In [72]:
num2label = {2 : 1 , 0 : 0 }
# data có label là 2 tương ứng positive còn 0 ứng với negative
# còn của model là {0: 'NEGATIVE', 1: 'POSITIVE'}
def convertlabel (samples) :
    return {"sentiment" : [num2label [num] for num in samples["sentiment"]] }
con_dataset = no_neutral_data.map (convertlabel , batched = True)
con_dataset

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 10968
    })
    validation: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 1510
    })
    test: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 2999
    })
})

In [74]:
def trans_data (samples) :
    return {"sentence" : [translate_vietnamese_to_english (sample ) for sample in  samples["sentence"] ] }

In [7]:
#train = con_dataset['train'].map (trans_data , batched = True)
val = con_dataset['validation'].map (trans_data , batched = True)
#test = con_dataset['test'].map (trans_data , batched = True)

NameError: name 'con_dataset' is not defined

###  LOAD MODEL

In [5]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

# inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
# with torch.no_grad():
#     logits = model(**inputs).logits

# predicted_class_id = logits.argmax().item()
# model.config.id2label[predicted_class_id]
# model.config.id2label

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [6]:
def tokenize(samples) :
    return tokenizer (samples['sentence'] , truncation = True, max_length = 256  )
# val_tokenize = val.map (tokenize , batched = True )
val_tokenize = con_dataset.map (tokenize , batched = True )


# val_tokenize = val.map (tokenize , batched = True )
# test_tokenize = test.map (tokenize , batched = True )

NameError: name 'con_dataset' is not defined

In [ ]:
val_data = val_tokenize.remove_columns (['sentence' , 'topic'])
val_data = val_data.rename_column ('sentiment' , 'labels')
val_data[0]

In [63]:
epochs = 10
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
val_dataloader = DataLoader (val_data, batch_size= 256, shuffle=True , collate_fn = data_collator )

metric_valid_acc = evaluate.load("accuracy")

metric_valid_f1 = evaluate.load("f1")

In [69]:
model.eval()
val_losses = []
prediction = []
for batch in tqdm ( val_dataloader ) :
    with torch.no_grad () :
        labels = batch.pop ("labels")
        logits = model.forward (**batch).logits

        criterion = nn.CrossEntropyLoss()
        #print (outputs.shape , labels.shape)
        loss = criterion(logits,labels)
        val_losses.append (loss.item())

        predict = torch.argmax (logits, dim = -1)
        prediction.append (predict)
        metric_valid_acc.add_batch (references=labels, predictions = predict)
        metric_valid_f1.add_batch (references=labels, predictions = predict )

val_acc = metric_valid_acc.compute ()['accuracy']
# if val_acc > best_val_acc :
#     torch.save(model.state_dict(), "Phobert_v2_best_model_full_data_{}".format (i))
#     best_val_acc = val_acc
#     i+=1
print ("Epoch : {}, Val Loss: {} , Validation  ACC Result : {} , Validation F1 Result :{}".format (epoch + 1, np.mean( val_losses) , val_acc , metric_valid_f1.compute (average="macro")['f1'] ))

  0%|          | 0/6 [00:00<?, ?it/s]

TypeError: Sequential.forward() got an unexpected keyword argument 'input_ids'

In [24]:
import numpy as np
print ("Val Loss: {} , Validation  ACC Result : {} , Validation F1 Result :{}".format ( np.mean( val_losses) , val_acc , metric_valid_f1.compute (average="macro")['f1'] ))

Val Loss: 0.39537401994069415 , Validation  ACC Result : 0.9139072847682119 , Validation F1 Result :0.9126381412173517


# Train adapter on Translation data

In [5]:
%%capture
!pip install -U sentence-transformers
!pip install pyvi

In [6]:
from sentence_transformers import SentenceTransformer
from pyvi.ViTokenizer import tokenize

### LOAD MODEL AND DATASET

### LOAD Vietnames embedding model and english embedding model. english will be used to creat embedding label (target), apply sentence transformer to this. vietnames will containt the SWITCH for convert vietnames to english later

In [7]:
vi_model =  AutoModel.from_pretrained('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base')
# eng_model = SentenceTransformer ('princeton-nlp/sup-simcse-roberta-base')
eng_model = AutoModel.from_pretrained ('distilbert-base-uncased-finetuned-sst-2-english')
# eng_model = AutoModel.from_pretrained ('sentence-transformers/all-mpnet-base-v2')

config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/542M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [8]:
vi_tok = AutoTokenizer.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")
eng_tok = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
# eng_tok = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2")

tokenizer_config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [9]:
embedding_en = eng_model.embeddings

### ADD ADAPTER TO VI MODEL

In [10]:
vi_model.embeddings.add_module (module = torch.nn.Sequential (
    nn.Dropout (0.1)  , 
    nn.Linear (768,768)  ) ,
    name = "dropout" )

### CREATE EMBEDDING TARGET FOR  ENGLISH

In [11]:
# LOAD DATA
en_vi = load_dataset("opus100", language_pair = "en-vi")
train_data = en_vi["train"].select (range (10000))
train_data

Generating test split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset opus100 downloaded and prepared to /root/.cache/huggingface/datasets/opus100/default-language_pair=en-vi/1.0.0/256f3196b69901fb0c79810ef468e2c4ed84fbd563719920b1ff1fdc750f7704. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Dataset({
    features: ['translation'],
    num_rows: 10000
})

In [65]:
# def embedding_language (samples) :
#     vi_tokenize = [tokenize(sentence['vi']) for sentence in samples['translation']]
#     eng_sen = [sample['en']  for sample in samples["translation"]]
#     with torch.no_grad () :
#         vi_emb = vi_model.encode ( vi_tokenize)
#         eng_emb = eng_model.encode(eng_sen)
#     return {"vi_emb" : vi_emb ,"en_emb" : eng_emb   }
            #, "en" : [eng_model ( sample['en'] ) for sample in samples["translation"]] }

In [66]:
# %%capture
# en_vi_emb = en_vi["train"].map (embedding_language , batched = True)
# len ( en_vi_emb[0]['vi_emb'] )

### THIS WILL TOKENIZE VIETNAM AND CREATE EMBEDDING TARGET FOR ENGLISH

In [12]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


In [13]:
# boi vi bi trung ten nen minh se phai tao 2 du lieu tokenize rieng le voi nhau
def tokenize_vi (samples) :
    vi_tokenize = [tokenize(sentence['vi']) for sentence in samples['translation']]
    return vi_tok(vi_tokenize, padding = "max_length", truncation= True , max_length = 100) 
### DATA KHÔNG CÙNG SHAPE.p
# def tokenize_en (samples) :
#     eng_sen = [sample['en']  for sample in samples["translation"]]
#     with torch.no_grad () :
#         eng_tokenizer = eng_tok (eng_sen, padding = "max_length", truncation= True , max_length = 100 , return_tensors = "pt")
#         eng_embedding = eng_model (**eng_tokenizer)
# #         embedding_list = [tensor.tolist() for tensor in eng_embedding]

#         sentence_embeddings = mean_pooling(eng_embedding, eng_tokenizer['attention_mask'])
#         embedding_list = sentence_embeddings.numpy().tolist ()
#     return {"en_emb" : embedding_list }


def tokenize_en (samples) :
    eng_sen = [sample['en']  for sample in samples["translation"]]
    with torch.no_grad () :
        eng_tokenizer = eng_tok (eng_sen, padding = "max_length", truncation= True , max_length = 100 , return_tensors = "pt")
        eng_embedding = embedding_en.forward (eng_tokenizer['input_ids'])
#         embedding_list = [tensor.tolist() for tensor in eng_embedding]

#         sentence_embeddings = mean_pooling(eng_embedding, eng_tokenizer['attention_mask'])
        print (eng_embedding.shape)
        embedding_list = eng_embedding.numpy().tolist ()
    return {"en_emb" : embedding_list }

In [14]:
vi_tok_data = train_data.map (tokenize_vi , batched = True  )

  0%|          | 0/10 [00:00<?, ?ba/s]

In [210]:
# import numpy as np
# a = np.array ( [len ( vi_tok_data[i]['input_ids'] ) for i in range (2000)] )
# a.min()

In [15]:
vi_en_tok_data = vi_tok_data.map (tokenize_en , batched = True , remove_columns = ["translation"])

  0%|          | 0/10 [00:00<?, ?ba/s]

torch.Size([1000, 100, 768])
torch.Size([1000, 100, 768])
torch.Size([1000, 100, 768])
torch.Size([1000, 100, 768])
torch.Size([1000, 100, 768])
torch.Size([1000, 100, 768])
torch.Size([1000, 100, 768])
torch.Size([1000, 100, 768])
torch.Size([1000, 100, 768])
torch.Size([1000, 100, 768])


#### SPLIT DATA

In [16]:
split_dataset = vi_en_tok_data.train_test_split (train_size = 0.9 ,seed = 42 )
split_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'en_emb'],
        num_rows: 9000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'en_emb'],
        num_rows: 1000
    })
})

### TRAIN ADAPTER

In [22]:
# class Adapter (nn.Module) :
#     def __init__(self, ):
#         super(Adapter, self).__init__()
#         self.adapter = nn.Linear (768 , 768)
    
#     def forward(self, vi_emb_stack):
#         fake_en_emb = self.adapter (vi_emb_stack) 
#         return fake_en_emb
# adapter = Adapter()

## Freeze Layers

In [18]:
vi_model = vi_model.embeddings

In [19]:
vi_model

RobertaEmbeddings(
  (word_embeddings): Embedding(64001, 768, padding_idx=1)
  (position_embeddings): Embedding(258, 768, padding_idx=1)
  (token_type_embeddings): Embedding(1, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (dropout): Sequential(
    (0): Dropout(p=0.1, inplace=False)
    (1): Linear(in_features=768, out_features=768, bias=True)
  )
)

In [20]:
for params in vi_model.parameters() :
    params.requires_grad = False

# for params in vi_model.embeddings.dropout.parameters() :
#     params.requires_grad = True
for params in vi_model.dropout.parameters() :
    params.requires_grad = True

In [21]:
vi_model.train()
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

def count_trainable_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print_trainable_parameters (vi_model)
count_trainable_params (vi_model)

trainable params: 590592 || all params: 49943808 || trainable%: 1.1825129553597515


590592

## Training Arguments

In [25]:
# def collate_tokenize(data):
#     text_batch = [element["text"] for element in data]
#     tokenized = tokenizer(text_batch, padding='longest', truncation=True, return_tensors='pt')
#     return tokenized

In [22]:
epochs = 3
#data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
train_dataloader = DataLoader (split_dataset["train"], batch_size= 128 , shuffle= False ,collate_fn=lambda x: x)
val_dataloader = DataLoader (split_dataset['test'], batch_size= 128 , shuffle= False   , collate_fn=lambda x: x )
accelerator = Accelerator ()
device = torch.device ("cuda") if torch.cuda.is_available() else torch.device ("cpu" )

step_to_train = epochs * len (val_dataloader)
optimizer = torch.optim.AdamW (vi_model.parameters () , lr = 1e-4)
scheduler = get_scheduler (name = "cosine" , 
                           optimizer = optimizer ,
                           num_warmup_steps=0,
                           num_training_steps = step_to_train
                          )

train_dataloader , val_dataloader ,vi_model , optimizer= accelerator.prepare (train_dataloader , val_dataloader , vi_model , optimizer)


In [23]:
split_dataset["train"]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'en_emb'],
    num_rows: 9000
})

### TRAINING AND EVALUATION

In [25]:
import numpy as np 
prediction = []
criterion = nn.MSELoss()
best_val_loss = 0
version = 0
for epoch in range (epochs) : 
    train_losses = []
    vi_model.train()
    for batch in tqdm ( train_dataloader ) :
#         translation = batch.pop ("translation")
#         print (translation)
#         eng_sen = translation['en'] 
#         print (eng_sen)
#         with torch.no_grad () :
#             eng_embedding = eng_model.encode(eng_sen)
#         en_emb_stack = torch.tensor ( eng_embedding ).to(device)


#         vi_emb = batch.pop ("vi_emb")
#         vi_emb_stack = torch.stack (vi_emb , dim = 1).float () .to(device)
        #FORWARD vimodel
        en_emb = [sample ['en_emb'] for sample in batch]
        en_emb = torch.tensor (en_emb).to(device)

        input_ids = [sample ['input_ids'] for sample in batch]
        input_ids = torch.tensor (input_ids).to(device)
        
#         token_type_ids = [sample ['token_type_ids'] for sample in batch]
#         token_type_ids = torch.tensor (token_type_ids).to(device)
        
#         attention_mask = [sample ['attention_mask'] for sample in batch]
#         attention_mask = torch.tensor (attention_mask).to(device)
        
        inputs = {"input_ids" : input_ids , "token_type_ids" : token_type_ids , "attention_mask" : attention_mask}
        #print (en_emb.shape)
        #batch = [{ 'input_ids' : sample['input_ids'] ,'token_type_ids' : sample['token_type_ids'] ,'attention_mask' : sample['attention_mask'] } for sample in batch ]
        
#         fake_en_emb =  vi_model(**inputs).pooler_output
        fake_en_emb =  vi_model.forward(inputs['input_ids'])
        loss =  criterion (fake_en_emb , en_emb)

        # BACKWARD ADAPTER
        accelerator.backward (loss)

        optimizer.step ()
        scheduler.step ()
        
        
        optimizer.zero_grad ()
        
        train_losses.append (loss.item())
    
            #print (outputs.shape , labels.shape)
    print ("TRAINING : Epoch : {}, TRAINING Loss: {}".format (epoch + 1, np.mean( train_losses) ) )
    
    vi_model.eval()
    val_losses = []
    for batch in  tqdm (val_dataloader)  :
        
        en_emb = [sample ['en_emb'] for sample in batch]
        en_emb = torch.tensor (en_emb).to(device)

        input_ids = [sample ['input_ids'] for sample in batch]
        input_ids = torch.tensor (input_ids).to(device)
        
#         token_type_ids = [sample ['token_type_ids'] for sample in batch]
#         token_type_ids = torch.tensor (token_type_ids).to(device)
        
#         attention_mask = [sample ['attention_mask'] for sample in batch]
#         attention_mask = torch.tensor (attention_mask).to(device)
        
        inputs = {"input_ids" : input_ids , "token_type_ids" : token_type_ids , "attention_mask" : attention_mask}
        #print (en_emb.shape)
        #batch = [{ 'input_ids' : sample['input_ids'] ,'token_type_ids' : sample['token_type_ids'] ,'attention_mask' : sample['attention_mask'] } for sample in batch ]
#         fake_en_emb =  vi_model(**inputs).pooler_output
        fake_en_emb =  vi_model.forward(inputs['input_ids'])
        loss =  criterion (fake_en_emb , en_emb)
        
        val_losses.append (loss.item())
    if np.mean( val_losses) > best_val_loss :
        torch.save(vi_model.state_dict(), "vietnam model {}".format (version))
        version +=1
        best_val_loss = np.mean( val_losses)
    print ("VALIDATION : Epoch : {}, Val Loss: {} ".format (epoch + 1, np.mean( val_losses) ))


  0%|          | 0/71 [00:00<?, ?it/s]

TRAINING : Epoch : 1, TRAINING Loss: 0.12190445365620331


  0%|          | 0/8 [00:00<?, ?it/s]

VALIDATION : Epoch : 1, Val Loss: 0.0959094287827611 


  0%|          | 0/71 [00:00<?, ?it/s]

TRAINING : Epoch : 2, TRAINING Loss: 0.08362030699639253


  0%|          | 0/8 [00:00<?, ?it/s]

VALIDATION : Epoch : 2, Val Loss: 0.07389085739850998 


  0%|          | 0/71 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Test Adapter on Sentiment Analysis

In [26]:
!pip install --quiet pyvi
from pyvi.ViTokenizer import tokenize

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Load Dataset

In [45]:
dataset_link = "uitnlp/vietnamese_students_feedback"
from datasets import load_dataset
dataset = load_dataset(dataset_link)
val_data = dataset["validation"]

  0%|          | 0/3 [00:00<?, ?it/s]

## Preprocess Data

In [28]:
def del_neutral (samples) :
    return samples['sentiment'] == 0 or samples['sentiment'] == 2
no_neutral_data = val_data.filter (del_neutral)
no_neutral_data

  0%|          | 0/2 [00:00<?, ?ba/s]

Dataset({
    features: ['sentence', 'sentiment', 'topic'],
    num_rows: 1510
})

In [29]:
num2label = {2 : 1 , 0 : 0 }
# data có label là 2 tương ứng positive còn 0 ứng với negative
# còn của model là {0: 'NEGATIVE', 1: 'POSITIVE'}
def convertlabel (samples) :
    return {"sentiment" : [num2label [num] for num in samples["sentiment"]] }
con_dataset = no_neutral_data.map (convertlabel , batched = True)
con_dataset

  0%|          | 0/2 [00:00<?, ?ba/s]

Dataset({
    features: ['sentence', 'sentiment', 'topic'],
    num_rows: 1510
})

## Embedding VietNam Data

In [140]:
# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
# model = AutoModel.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

In [141]:
# vietnam_model =  SentenceTransformer('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base')

In [9]:
# model = AutoModel.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
#vi_tok = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
# the model will be english but the tokenizer and embedding is vietnames

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [10]:
# def embedding_language (samples) :
#     vi_tokenize = [tokenize(sentence) for sentence in samples['sentence']]
#     with torch.no_grad () :
#         vi_emb = vietnam_model.encode ( vi_tokenize)
#     return {"vi_emb" : vi_emb}
            #, "en" : [eng_model ( sample['en'] ) for sample in samples["translation"]] }

In [11]:
# emb_data = con_dataset.map (embedding_language , batched = True)

In [30]:
vi_tok = AutoTokenizer.from_pretrained ("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")

In [31]:
def tokenize_language (samples) :
    vi_tokenize = [tokenize(sentence) for sentence in samples['sentence']]
    return vi_tok(vi_tokenize, padding = "max_length", truncation= True , max_length = 100) 
            #, "en" : [eng_model ( sample['en'] ) for sample in samples["translation"]] 

In [32]:
con_dataset

Dataset({
    features: ['sentence', 'sentiment', 'topic'],
    num_rows: 1510
})

In [33]:
input_dataset = con_dataset.map (tokenize_language , batched = True , remove_columns = ['sentence', 'topic'])
input_dataset

  0%|          | 0/2 [00:00<?, ?ba/s]

Dataset({
    features: ['sentiment', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1510
})

## Forward VIE embedding through adapter to create fake en_emb

In [24]:
# def forward_adapter (samples) :
#     tensor_samples = torch.tensor ( samples["vi_emb"] ).to(device)
#     fake_en_emb =  [ adapter (tensor_sample) for tensor_sample in tensor_samples ] 
#     return {"en_emb" :  fake_en_emb}

In [25]:
# en_emb_data = emb_data.map (forward_adapter , batched = True , remove_columns = ["sentence" , "topic" , "vi_emb"])

  0%|          | 0/2 [00:00<?, ?ba/s]

In [26]:
# en_emb_data

Dataset({
    features: ['sentiment', 'en_emb'],
    num_rows: 1510
})

## classification

In [34]:
# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english").to(device)

In [35]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [38]:
model.distilbert.add_module (module = vi_model , name = "embeddings")

In [39]:
vi_model

RobertaEmbeddings(
  (word_embeddings): Embedding(64001, 768, padding_idx=1)
  (position_embeddings): Embedding(258, 768, padding_idx=1)
  (token_type_embeddings): Embedding(1, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (dropout): Sequential(
    (0): Dropout(p=0.1, inplace=False)
    (1): Linear(in_features=768, out_features=768, bias=True)
  )
)

In [40]:
model.distilbert.embeddings

RobertaEmbeddings(
  (word_embeddings): Embedding(64001, 768, padding_idx=1)
  (position_embeddings): Embedding(258, 768, padding_idx=1)
  (token_type_embeddings): Embedding(1, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (dropout): Sequential(
    (0): Dropout(p=0.1, inplace=False)
    (1): Linear(in_features=768, out_features=768, bias=True)
  )
)

In [41]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Sequential(
        (0): Dropout(p=0.1, inplace=False)
        (1): Linear(in_features=768, out_features=768, bias=True)
      )
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)

In [42]:
data_collator = DataCollatorWithPadding(tokenizer= vi_tok)

val_dataloader = DataLoader (input_dataset, batch_size= 128, shuffle=True ,  collate_fn = data_collator )
#Metric
accelerator = Accelerator ()
val_dataloader ,model= accelerator.prepare ( val_dataloader , model )


metric_valid_acc = evaluate.load("accuracy")

metric_valid_f1 = evaluate.load("f1")

In [32]:
# model.eval()
# val_losses = []
# prediction = []
# criterion = nn.CrossEntropyLoss()
# for batch in tqdm ( val_dataloader ) :
#     with torch.no_grad () :
#         labels = batch.pop ("sentiment").to(device)
#         en_emb = batch.pop ("en_emb")
#         en_emb_stack = torch.stack (en_emb , dim = 1 ).float ().to(device)
#         with torch.no_grad () :
#             output = model.pre_classifier.forward (en_emb_stack)
#             output = model.classifier.forward (output)
#             logits = model.dropout.forward(output)
        
#         #print (outputs.shape , labels.shape)
#         loss = criterion(logits,labels)
#         predict = torch.argmax (logits, dim = -1)
        
        
#         val_losses.append (loss.item())
#         prediction.append (predict)
#         metric_valid_acc.add_batch (references=labels, predictions = predict)
#         metric_valid_f1.add_batch (references=labels, predictions = predict )

# val_acc = metric_valid_acc.compute ()['accuracy']
# # if val_acc > best_val_acc :
# #     torch.save(model.state_dict(), "Phobert_v2_best_model_full_data_{}".format (i))
# #     best_val_acc = val_acc
# #     i+=1
# print ("Epoch : {}, Val Loss: {} , Validation  ACC Result : {} , Validation F1 Result :{}".format (epoch + 1, np.mean( val_losses) , val_acc , metric_valid_f1.compute (average="macro")['f1'] ))

  0%|          | 0/755 [00:00<?, ?it/s]

Exception ignored in: <function tqdm.__del__ at 0x7f4a7b3904c0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/tqdm/std.py", line 1149, in __del__
    self.close()
  File "/opt/conda/lib/python3.10/site-packages/tqdm/notebook.py", line 278, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm' object has no attribute 'disp'


Epoch : 1, Val Loss: 0.2886639424579897 , Validation  ACC Result : 0.8933774834437086 , Validation F1 Result :0.8912128573864148


In [35]:
# torch.save(adapter.state_dict(), "adapter " )

In [46]:
model.eval()
val_losses = []
prediction = []
epoch = 0
criterion = nn.CrossEntropyLoss()
for batch in tqdm ( val_dataloader ) :
    with torch.no_grad () :
        batch.pop ("token_type_ids")
        labels = batch.pop ("sentiment").to("cuda")
        logits = model(**batch).logits
        #print (outputs.shape , labels.shape)
        loss = criterion(logits,labels)
        predict = torch.argmax (logits, dim = -1)
        
        
        val_losses.append (loss.item())
        prediction.append (predict)
        metric_valid_acc.add_batch (references=labels, predictions = predict)
        metric_valid_f1.add_batch (references=labels, predictions = predict )

val_acc = metric_valid_acc.compute ()['accuracy']
# if val_acc > best_val_acc :
#     torch.save(model.state_dict(), "Phobert_v2_best_model_full_data_{}".format (i))
#     best_val_acc = val_acc
#     i+=1
print ("Epoch : {}, Val Loss: {} , Validation  ACC Result : {} , Validation F1 Result :{}".format (epoch + 1, np.mean( val_losses) , val_acc , metric_valid_f1.compute (average="macro")['f1'] ))

  0%|          | 0/12 [00:00<?, ?it/s]

Epoch : 1, Val Loss: 2.183247764905294 , Validation  ACC Result : 0.47218543046357614 , Validation F1 Result :0.34134790887114563


In [22]:
val_acc

0.5

## TEST 

In [9]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForQuestionAnswering

processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")

img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg' 
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

question = "how many dogs are in the picture?"
inputs = processor(raw_image, question, return_tensors="pt")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

1


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1128: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [41]:
inputs

{'pixel_values': tensor([[[[ 0.8647,  0.9230,  0.9376,  ...,  1.7552,  1.7552,  1.7552],
          [ 0.9084,  0.9376,  0.9522,  ...,  1.7552,  1.7552,  1.7552],
          [ 0.9376,  0.9376,  0.9668,  ...,  1.7552,  1.7552,  1.7552],
          ...,
          [-0.7850, -0.7850, -0.7266,  ..., -0.3178, -0.2740, -0.3616],
          [-0.7558, -0.7558, -0.7412,  ..., -0.3178, -0.3616, -0.4346],
          [-0.7558, -0.7704, -0.7850,  ..., -0.3616, -0.4346, -0.4784]],

         [[ 1.2194,  1.2495,  1.2795,  ...,  1.8948,  1.8948,  1.8948],
          [ 1.2344,  1.2645,  1.2945,  ...,  1.8948,  1.8948,  1.8948],
          [ 1.2495,  1.2795,  1.3095,  ...,  1.8948,  1.8948,  1.8948],
          ...,
          [-0.5965, -0.5965, -0.5515,  ..., -0.4014, -0.3264, -0.4164],
          [-0.5665, -0.5665, -0.5515,  ..., -0.3864, -0.4164, -0.4914],
          [-0.5665, -0.5815, -0.5965,  ..., -0.4164, -0.4764, -0.5365]],

         [[ 1.2927,  1.3211,  1.3496,  ...,  1.9753,  1.9753,  1.9753],
          [ 1

In [46]:
model.vision_model.forward (inputs["pixel_values"]).keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [11]:
model.text_encoder = vi_model.embeddings 

In [13]:
model.text_encoder

RobertaEmbeddings(
  (word_embeddings): Embedding(64001, 768, padding_idx=1)
  (position_embeddings): Embedding(258, 768, padding_idx=1)
  (token_type_embeddings): Embedding(1, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [12]:
model

BlipForQuestionAnswering(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-11): 12 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((768,), eps=1e-05, e

In [39]:
model.text_decoder

BlipTextLMHeadModel(
  (bert): BlipTextModel(
    (embeddings): BlipTextEmbeddings(
      (word_embeddings): Embedding(30524, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): BlipTextEncoder(
      (layer): ModuleList(
        (0-11): 12 x BlipTextLayer(
          (attention): BlipTextAttention(
            (self): BlipTextSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): BlipTextSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
     